In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sentencepiece as spm
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

import re
import os
import random
import math
import gensim

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

## 1. 데이터 다운로드

In [2]:
data = pd.read_csv(os.getenv('HOME')+'/aiffel/transformer_chatbot/data/ChatbotData .csv')
data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [3]:
questions = data['Q']
answers = data['A']

In [4]:
questions.head()

0             12시 땡!
1        1지망 학교 떨어졌어
2       3박4일 놀러가고 싶다
3    3박4일 정도 놀러가고 싶다
4            PPL 심하네
Name: Q, dtype: object

In [5]:
answers.head()

0     하루가 또 가네요.
1      위로해 드립니다.
2    여행은 언제나 좋죠.
3    여행은 언제나 좋죠.
4     눈살이 찌푸려지죠.
Name: A, dtype: object

## 2. 데이터 정제

In [6]:
def preprocess_sentence(sentence):
    
    sentence = sentence.lower()
    sentence = re.sub(r"[^a-zA-Z가-힣0-9?.!,]+", " ", sentence)
    
    return sentence

## 3. 데이터 토큰화

In [7]:
from konlpy.tag import Mecab

mecab = Mecab()

def build_corpus(ssd, tsd):
    mecab_s_corpus, mecab_t_corpus = [], []
    mecab_s_len_list, mecab_t_len_list = [], []
    
    for s, t in zip(ssd, tsd):
        s = mecab.morphs(preprocess_sentence(s))
        t = mecab.morphs(preprocess_sentence(t))
        
        mecab_s_corpus.append(s)
        mecab_t_corpus.append(t)
        
        mecab_s_len_list.append(len(s))
        mecab_t_len_list.append(len(t))

    mecab_num_tokens = mecab_s_len_list + mecab_t_len_list
    
    mean_len = np.mean(mecab_num_tokens)
    max_len = np.max(mecab_num_tokens)
    mid_len = np.median([mean_len, max_len])
    
    s_corpus, t_corpus = [], []
    for q, a in zip(mecab_s_corpus, mecab_t_corpus):
        if len(q) <= mid_len and len(a) <= mid_len:
            if q not in s_corpus and a not in t_corpus:
                s_corpus.append(q)
                t_corpus.append(a)
    
    return s_corpus, t_corpus

In [8]:
que_corpus, ans_corpus = build_corpus(questions, answers)

In [9]:
que_corpus[:5]

[['12', '시', '땡', '!'],
 ['1', '지망', '학교', '떨어졌', '어'],
 ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'],
 ['ppl', '심하', '네'],
 ['sd', '카드', '망가졌', '어']]

In [10]:
ans_corpus[:5]

[['하루', '가', '또', '가', '네요', '.'],
 ['위로', '해', '드립니다', '.'],
 ['여행', '은', '언제나', '좋', '죠', '.'],
 ['눈살', '이', '찌푸려', '지', '죠', '.'],
 ['다시', '새로', '사', '는', '게', '마음', '편해요', '.']]

## 4. Augmentation

In [11]:
kobin = os.getenv('HOME') + '/aiffel/transformer_chatbot/data/ko.bin'
word2vec = gensim.models.Word2Vec.load(kobin)

In [12]:
def lexical_sub(sentence, word2vec):
    try:
        _from = random.choice(sentence)
        _to = word2vec.most_similar(_from)[0][0]
    except:
        return sentence
    
    res = []
    for x in sentence:
        if x is _from: res.append(_to)
        else: res.append(x)

    return res

In [13]:
arg_que_corpus = [lexical_sub(x, word2vec) for x in que_corpus]
arg_ans_corpus = [lexical_sub(x, word2vec) for x in ans_corpus]

/tmp/ipykernel_330/1225969195.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  _to = word2vec.most_similar(_from)[0][0]


In [14]:
for i in range(5):
    print(f"Q : {' '.join(que_corpus[i])} / {' '.join(arg_que_corpus[i])}")
    print(f"A : {' '.join(ans_corpus[i])} / {' '.join(arg_ans_corpus[i])}")

Q : 12 시 땡 ! / 12 시 땡 캐치
A : 하루 가 또 가 네요 . / 일주일 가 또 가 네요 .
Q : 1 지망 학교 떨어졌 어 / 1 지망 학교의 떨어졌 어
A : 위로 해 드립니다 . / 위로 해 드립니다 는데
Q : 3 박 4 일 놀 러 가 고 싶 다 / 3 박 4 일 놀 러 가 고 싶 다
A : 여행 은 언제나 좋 죠 . / 여행 은 항상 좋 죠 .
Q : ppl 심하 네 / ppl 심하 네
A : 눈살 이 찌푸려 지 죠 . / 눈살 이 찌푸려 꼼짝 죠 .
Q : sd 카드 망가졌 어 / sd 단말기 망가졌 어
A : 다시 새로 사 는 게 마음 편해요 . / 다시 새로 타 는 게 마음 편해요 .


In [15]:
que_corpus = que_corpus + arg_que_corpus + que_corpus
ans_corpus = ans_corpus + ans_corpus + arg_ans_corpus

In [16]:
len(que_corpus), len(ans_corpus)

(22911, 22911)

In [18]:
ans_corpus[:3]

[['하루', '가', '또', '가', '네요', '.'],
 ['위로', '해', '드립니다', '.'],
 ['여행', '은', '언제나', '좋', '죠', '.']]

## 5. 데이터 벡터화

In [19]:
ans_corpus = [["<start>"] + ans + ["<end>"] for ans in ans_corpus]
ans_corpus[:3]

[['<start>', '하루', '가', '또', '가', '네요', '.', '<end>'],
 ['<start>', '위로', '해', '드립니다', '.', '<end>'],
 ['<start>', '여행', '은', '언제나', '좋', '죠', '.', '<end>']]

In [20]:
wdata = que_corpus + ans_corpus

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=None, filters=' ', oov_token='<unk>')
tokenizer.fit_on_texts(wdata)
tensor = tokenizer.texts_to_sequences(wdata)
tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

In [21]:
VOCAB_SIZE = len(tokenizer.index_word) + 2
VOCAB_SIZE

7129

In [22]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    if idx >= 5: 
        break

1 : <unk>
2 : .
3 : <start>
4 : <end>
5 : 이


In [23]:
enc_train, dec_train = tensor[:22911], tensor[22911:]
enc_train.shape, dec_train.shape

((22911, 25), (22911, 25))